In [254]:
import pandas as pd
from datetime import datetime

In [255]:
df=pd.read_csv('Datasource/Cases Report.csv')
df.head()

,Outcome,Age,Gender,Reporting PHU,SpecimenDate,CaseReported Date,PHUCity,TestReported Date,CaseAcquisition info,AccurateEpisode Dt,PHU Address,PHU Website,OutbreakRelated,PHU Latitude,PHU Longitude,PHU Postal Code
0,Resolved,80s,FEMALE,Toronto Public Health,9/24/2020,2/6/2021,Toronto,2/6/2021,OB,1/26/2021,"277 Victoria Street, 5th Floor",www.toronto.ca/community-people/health-wellnes...,Yes,43.656591,-79.379358,M5B 1W2
1,Resolved,<20,MALE,Peel Public Health,12/24/2020,2/1/2021,Mississauga,2/1/2021,CC,1/29/2021,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4
2,Resolved,20s,MALE,Lambton Public Health,9/23/2020,1/15/2021,Point Edward,1/14/2021,CC,1/10/2021,160 Exmouth Street,www.lambtonhealth.on.ca,Yes,42.986416,-82.404808,N7T 7Z6
3,Resolved,20s,MALE,"Region of Waterloo, Public Health",9/20/2020,12/29/2020,Waterloo,9/23/2020,NO KNOWN EPI LINK,9/20/2020,99 Regina Street South,www.regionofwaterloo.ca,NaN,43.462876,-80.520913,N2J 4V3
4,Resolved,20s,FEMALE,Ottawa Public Health,9/21/2020,12/24/2020,Ottawa,9/22/2020,CC,9/21/2020,100 Constellation Drive,www.ottawapublichealth.ca,NaN,45.345665,-75.763912,K2G 6J8


In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28500 entries, 0 to 28499
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Outcome               28500 non-null  object 
 1   Age                   28500 non-null  object 
 2   Gender                28500 non-null  object 
 3   Reporting PHU         28500 non-null  object 
 4   SpecimenDate          28292 non-null  object 
 5   CaseReported Date     28500 non-null  object 
 6   PHUCity               28500 non-null  object 
 7   TestReported Date     28082 non-null  object 
 8   CaseAcquisition info  28500 non-null  object 
 9   AccurateEpisode Dt    28500 non-null  object 
 10  PHU Address           28500 non-null  object 
 11  PHU Website           28500 non-null  object 
 12  OutbreakRelated       8710 non-null   object 
 13  PHU Latitude          28500 non-null  float64
 14  PHU Longitude         28500 non-null  float64
 15  PHU Postal Code    

In [257]:
df.isnull().sum()

Outcome                     0
Age                         0
Gender                      0
Reporting PHU               0
SpecimenDate              208
CaseReported Date           0
PHUCity                     0
TestReported Date         418
CaseAcquisition info        0
AccurateEpisode Dt          0
PHU Address                 0
PHU Website                 0
OutbreakRelated         19790
PHU Latitude                0
PHU Longitude               0
PHU Postal Code             0
dtype: int64

In [258]:
df['CaseAcquisition info'].unique()

array(['OB', 'CC', 'NO KNOWN EPI LINK', 'TRAVEL', 'MISSING INFORMATION',
       'UNSPECIFIED EPI LINK'], dtype=object)

In [259]:
df.loc[df['CaseAcquisition info']=='UNSPECIFIED EPI LINK','CaseAcquisition info'] = 'NO KNOWN EPI LINK'
df.loc[df['CaseAcquisition info']=='MISSING INFORMATION','CaseAcquisition info'] = 'NOT REPORTED'

# Outcome, Age, Gender

In [260]:
df['Outcome'].value_counts()

Resolved        27192
Fatal            1307
Not Resolved        1
Name: Outcome, dtype: int64

In [261]:
df.loc[df['Outcome']=='Resolved','Outcome'] = 'Recovered'
df.loc[df['Outcome']=='Fatal','Outcome'] = 'Deceased'
df.loc[df['Outcome']=='Not Resolved','Outcome'] = 'Active'

In [262]:
df['Age'].value_counts()

20s    5899
30s    4363
50s    4280
40s    3952
60s    2879
<20    2556
80s    1737
70s    1699
90+    1135
Name: Age, dtype: int64

In [263]:
df.loc[df['Age']=='20s','Age'] = '20-29'
df.loc[df['Age']=='30s','Age'] = '30-39'
df.loc[df['Age']=='40s','Age'] = '40-49'
df.loc[df['Age']=='50s','Age'] = '50-59'
df.loc[df['Age']=='60s','Age'] = '60-69'
df.loc[df['Age']=='70s','Age'] = '70-79'
df.loc[df['Age']=='80s','Age'] = '80+'
df.loc[df['Age']=='90+','Age'] = '80+'

In [264]:
df.columns

Index(['Outcome', 'Age', 'Gender', 'Reporting PHU', 'SpecimenDate',
       'CaseReported Date', 'PHUCity', 'TestReported Date',
       'CaseAcquisition info', 'AccurateEpisode Dt', 'PHU Address',
       'PHU Website', 'OutbreakRelated', 'PHU Latitude', 'PHU Longitude',
       'PHU Postal Code'],
      dtype='object')

In [265]:
df['Gender'].value_counts()

FEMALE            14470
MALE              14006
UNSPECIFIED          21
GENDER DIVERSE        3
Name: Gender, dtype: int64

In [266]:
df['Gender']=df['Gender'].str.lower()

In [267]:
df[df.duplicated(keep='first')]

,Outcome,Age,Gender,Reporting PHU,SpecimenDate,CaseReported Date,PHUCity,TestReported Date,CaseAcquisition info,AccurateEpisode Dt,PHU Address,PHU Website,OutbreakRelated,PHU Latitude,PHU Longitude,PHU Postal Code
105,Recovered,<20,female,Hamilton Public Health Services,10/6/2020,10/9/2020,Hamilton,10/9/2020,CC,10/6/2020,"110 King St. West, 2nd Floor",www.hamilton.ca/publichealth,NaN,43.257631,-79.871341,L8P 4S6
115,Recovered,20-29,female,Peel Public Health,10/3/2020,10/9/2020,Mississauga,10/9/2020,CC,10/3/2020,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4
134,Recovered,20-29,male,Peel Public Health,10/3/2020,10/9/2020,Mississauga,10/9/2020,CC,10/3/2020,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4
162,Recovered,30-39,female,Peel Public Health,10/7/2020,10/9/2020,Mississauga,10/9/2020,CC,10/7/2020,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4
197,Recovered,<20,female,Hamilton Public Health Services,10/6/2020,10/8/2020,Hamilton,10/8/2020,OB,10/4/2020,"110 King St. West, 2nd Floor",www.hamilton.ca/publichealth,Yes,43.257631,-79.871341,L8P 4S6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,Recovered,30-39,female,Peel Public Health,3/28/2020,3/30/2020,Mississauga,3/30/2020,CC,3/23/2020,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4
27102,Recovered,40-49,male,York Region Public Health Services,3/26/2020,3/30/2020,Newmarket,3/30/2020,NO KNOWN EPI LINK,3/19/2020,17250 Yonge Street,www.york.ca/wps/portal/yorkhome/health/,NaN,44.048023,-79.480239,L3Y 6Z1
27509,Recovered,20-29,female,Peel Public Health,3/25/2020,3/28/2020,Mississauga,3/28/2020,CC,3/24/2020,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4
27720,Recovered,60-69,female,Peel Public Health,3/21/2020,3/27/2020,Mississauga,3/27/2020,CC,3/14/2020,7120 Hurontario Street,www.peelregion.ca/health/,NaN,43.647471,-79.708893,L5W 1N4


# SpecimenDate, TestReported Date

In [268]:
df['SpecimenDate'].value_counts()

4/15/2020     404
4/16/2020     389
4/13/2020     383
4/11/2020     356
4/17/2020     336
             ... 
2/9/2021        1
10/16/2020      1
10/15/2020      1
3/3/2020        1
12/15/2020      1
Name: SpecimenDate, Length: 242, dtype: int64

In [269]:
df['SpecimenDate'] =  pd.to_datetime(df['SpecimenDate'], format='%m/%d/%Y')
df['SpecimenDate']

0       2020-09-24
1       2020-12-24
2       2020-09-23
3       2020-09-20
4       2020-09-21
           ...    
28495   2020-02-29
28496   2020-02-28
28497   2020-02-28
28498   2020-02-28
28499   2020-02-29
Name: SpecimenDate, Length: 28500, dtype: datetime64[ns]

In [270]:
df['SpecimenDate'] = df['SpecimenDate'].fillna(df.groupby('Reporting PHU')['SpecimenDate'].transform('median'))
df['SpecimenDate']

0       2020-09-24
1       2020-12-24
2       2020-09-23
3       2020-09-20
4       2020-09-21
           ...    
28495   2020-02-29
28496   2020-02-28
28497   2020-02-28
28498   2020-02-28
28499   2020-02-29
Name: SpecimenDate, Length: 28500, dtype: datetime64[ns]

In [271]:
df['SpecimenDate'] = pd.to_datetime(df['SpecimenDate']).dt.date
df['SpecimenDate']

0        2020-09-24
1        2020-12-24
2        2020-09-23
3        2020-09-20
4        2020-09-21
            ...    
28495    2020-02-29
28496    2020-02-28
28497    2020-02-28
28498    2020-02-28
28499    2020-02-29
Name: SpecimenDate, Length: 28500, dtype: object

In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28500 entries, 0 to 28499
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Outcome               28500 non-null  object 
 1   Age                   28500 non-null  object 
 2   Gender                28500 non-null  object 
 3   Reporting PHU         28500 non-null  object 
 4   SpecimenDate          28500 non-null  object 
 5   CaseReported Date     28500 non-null  object 
 6   PHUCity               28500 non-null  object 
 7   TestReported Date     28082 non-null  object 
 8   CaseAcquisition info  28500 non-null  object 
 9   AccurateEpisode Dt    28500 non-null  object 
 10  PHU Address           28500 non-null  object 
 11  PHU Website           28500 non-null  object 
 12  OutbreakRelated       8710 non-null   object 
 13  PHU Latitude          28500 non-null  float64
 14  PHU Longitude         28500 non-null  float64
 15  PHU Postal Code    

In [273]:
df['TestReported Date'] =  pd.to_datetime(df['TestReported Date'], format='%m/%d/%Y')
df['TestReported Date'] = df['TestReported Date'].fillna(df.groupby('Reporting PHU')['TestReported Date'].transform('median'))
df['TestReported Date'] = pd.to_datetime(df['TestReported Date']).dt.date

In [274]:
df.isnull().sum()

Outcome                     0
Age                         0
Gender                      0
Reporting PHU               0
SpecimenDate                0
CaseReported Date           0
PHUCity                     0
TestReported Date           0
CaseAcquisition info        0
AccurateEpisode Dt          0
PHU Address                 0
PHU Website                 0
OutbreakRelated         19790
PHU Latitude                0
PHU Longitude               0
PHU Postal Code             0
dtype: int64

In [275]:
df['CaseReported Date'] =  pd.to_datetime(df['CaseReported Date'], format='%m/%d/%Y').dt.date
df['AccurateEpisode Dt'] =  pd.to_datetime(df['AccurateEpisode Dt'], format='%m/%d/%Y').dt.date

# OutbreakRelated

In [276]:
df['OutbreakRelated'].unique()

array(['Yes', nan], dtype=object)

In [277]:
bool_series = pd.isnull(df['OutbreakRelated'])
df[bool_series].groupby('CaseAcquisition info')['CaseAcquisition info'].count().sort_values(ascending=True).reset_index(name='Count')

,CaseAcquisition info,Count
0,OB,246
1,NOT REPORTED,250
2,TRAVEL,1721
3,NO KNOWN EPI LINK,6423
4,CC,11150


In [278]:
bool_series = pd.notnull(df['OutbreakRelated'])
df[bool_series].groupby('CaseAcquisition info')['CaseAcquisition info'].count().sort_values(ascending=True).reset_index(name='Count')

,CaseAcquisition info,Count
0,TRAVEL,2
1,NOT REPORTED,4
2,NO KNOWN EPI LINK,91
3,CC,393
4,OB,8220


In [279]:
pd.options.mode.chained_assignment = 'warn'
df.loc[df['CaseAcquisition info']=='OB','OutbreakRelated'] = df.loc[df['CaseAcquisition info']=='OB','OutbreakRelated'].fillna('Yes')

In [280]:
df[df['CaseAcquisition info']=='OB']['OutbreakRelated'].unique()

array(['Yes'], dtype=object)

In [281]:
df['OutbreakRelated'] = df['OutbreakRelated'].fillna('No')

In [282]:
df.isnull().sum()

Outcome                 0
Age                     0
Gender                  0
Reporting PHU           0
SpecimenDate            0
CaseReported Date       0
PHUCity                 0
TestReported Date       0
CaseAcquisition info    0
AccurateEpisode Dt      0
PHU Address             0
PHU Website             0
OutbreakRelated         0
PHU Latitude            0
PHU Longitude           0
PHU Postal Code         0
dtype: int64

In [283]:
df['CreatedDate']=pd.datetime.now()
df['UpdatedDate']=pd.datetime.now()

C:\Users\ngphv\AppData\Local\Temp\ipykernel_22220\943213306.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df['CreatedDate']=pd.datetime.now()
C:\Users\ngphv\AppData\Local\Temp\ipykernel_22220\943213306.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df['UpdatedDate']=pd.datetime.now()


In [284]:
df.head()

,Outcome,Age,Gender,Reporting PHU,SpecimenDate,CaseReported Date,PHUCity,TestReported Date,CaseAcquisition info,AccurateEpisode Dt,PHU Address,PHU Website,OutbreakRelated,PHU Latitude,PHU Longitude,PHU Postal Code,CreatedDate,UpdatedDate
0,Recovered,80+,female,Toronto Public Health,2020-09-24,2021-02-06,Toronto,2021-02-06,OB,2021-01-26,"277 Victoria Street, 5th Floor",www.toronto.ca/community-people/health-wellnes...,Yes,43.656591,-79.379358,M5B 1W2,2022-11-23 12:23:21.086731,2022-11-23 12:23:21.088744
1,Recovered,<20,male,Peel Public Health,2020-12-24,2021-02-01,Mississauga,2021-02-01,CC,2021-01-29,7120 Hurontario Street,www.peelregion.ca/health/,No,43.647471,-79.708893,L5W 1N4,2022-11-23 12:23:21.086731,2022-11-23 12:23:21.088744
2,Recovered,20-29,male,Lambton Public Health,2020-09-23,2021-01-15,Point Edward,2021-01-14,CC,2021-01-10,160 Exmouth Street,www.lambtonhealth.on.ca,Yes,42.986416,-82.404808,N7T 7Z6,2022-11-23 12:23:21.086731,2022-11-23 12:23:21.088744
3,Recovered,20-29,male,"Region of Waterloo, Public Health",2020-09-20,2020-12-29,Waterloo,2020-09-23,NO KNOWN EPI LINK,2020-09-20,99 Regina Street South,www.regionofwaterloo.ca,No,43.462876,-80.520913,N2J 4V3,2022-11-23 12:23:21.086731,2022-11-23 12:23:21.088744
4,Recovered,20-29,female,Ottawa Public Health,2020-09-21,2020-12-24,Ottawa,2020-09-22,CC,2020-09-21,100 Constellation Drive,www.ottawapublichealth.ca,No,45.345665,-75.763912,K2G 6J8,2022-11-23 12:23:21.086731,2022-11-23 12:23:21.088744


In [285]:
df.columns

Index(['Outcome', 'Age', 'Gender', 'Reporting PHU', 'SpecimenDate',
       'CaseReported Date', 'PHUCity', 'TestReported Date',
       'CaseAcquisition info', 'AccurateEpisode Dt', 'PHU Address',
       'PHU Website', 'OutbreakRelated', 'PHU Latitude', 'PHU Longitude',
       'PHU Postal Code', 'CreatedDate', 'UpdatedDate'],
      dtype='object')

In [286]:
df.columns=['Outcome', 'Age', 'Gender', 'ReportingPHU', 'SpecimenDate',
       'CaseReportedDate', 'PHUCity', 'TestReportedDate',
       'CaseAcquisitionInfo', 'AccurateEpisodeDt', 'PHUAddress',
       'PHUWebsite', 'OutbreakRelated', 'PHULatitude', 'PHULongitude',
       'PHUPostalCode','CreatedDate', 'UpdatedDate']
df.to_excel('CasesReport.xlsx',index=False)